# Sudoku

A popular number game/puzzle whose objective is to fill each of the empty cells in a grid with the correct number using the given initial clues. The classic sudoku game involves a 9x9 grid in which the grid is divided into 9 blocks, and each block contains 9 cells. 

## Solving Sudoku Puzzle

In order to solve a sudoku puzzle, every empty cell must be filled without violating the following constraints.
1. Each row must contain the numbers without repetitions
2. Each column must contain the numbers without repetitions
3. Each block must contain the numbers without repetitiions

One of the obvious ways to solve sudoku is to use a backtracking algorithm which tries all the possible combination of the board. This, howevers, can be inefficient. 

Therefore, in this project, I will be using Linear Optimization algorithm to solve sudoku puzzles.

In [1]:
import numpy as np

In [2]:
class Sudoku:

    def __init__(self, n = 9, k = 45):
        # initialize
        self.n = n
        self.k = k
        self.puzzle = self.get_puzzle()

    def get_puzzle(self):
        completed_boards = []
        board = np.zeros((self.n, self.n))
        # randomly fill only top-left block of the board to avoid the same board every time 
        numbers = np.arange(1, self.n + 1)
        np.random.shuffle(numbers)
        sqrt_n = (int) (np.sqrt(self.n))
        # simply go over the block and place the numbers since there is no constraint yet
        for i in range(sqrt_n):
            for j in range(sqrt_n):
                board[i, j] = numbers[sqrt_n*i + j]
        # solve the partially filled board for a complete board
        self.solve_backtracking(board, completed_boards, need_one_solution = True)
        # get the completed board
        completed_board = completed_boards[0]
        # use the completed board to generate a puzzle 
        # this guarantees at least one solution
        puzzle = np.copy(completed_board)
        # remove k random cells from the completed board to generate puzzle
        count = 0
        while (count < self.k):
            row = np.random.randint(self.n)
            col = np.random.randint(self.n)
            if (puzzle[row, col] != 0):
                puzzle[row, col] = 0
                count += 1
        return puzzle

    def solve_backtracking(self, board, solutions, need_one_solution = False):
        # stop in case only one solution is needed
        if (need_one_solution and len(solutions) == 1): return
        # traverse the board
        for row in range(self.n):
            for col in range(self.n):
                if (board[row, col] == 0):
                    # try to find the number to fill
                    for number in range(1, self.n + 1):
                        if (self.valid(row, col, number, board)):
                            board[row, col] = number
                            self.solve_backtracking(board, solutions, need_one_solution)
                            board[row, col] = 0
                    return
        # collect the solutions
        solutions.append(np.copy(board).astype(int))

    def solve_linear_optimization(self):
        # to be implemented
        pass    

    def valid(self, row, col, number, board):
        # generalize problem to work with nxn
        sqrt_n = (int) (np.sqrt(self.n))
        # check if number is in the given row
        in_row = number in board[row, :]
        # check if number is in the given col
        in_col = number in board[:, col]
        # get starting points of the block where the given row and col are in
        i = row - (row % sqrt_n)
        j = col - (col % sqrt_n)
        # check if number is in the given block
        in_block = number in board[i:(i + sqrt_n), j:(j + sqrt_n)]
        # valid if number is neither in row, col, nor block
        return not (in_row or in_col or in_block)

    def check(self, board):
        # traverse the board
        for row in range(self.n):
            for col in range(self.n):
                # save and remove the number from the cell
                number = board[row, col]
                board[row, col] = 0
                # check if the board is still valid if the number is placed back to the cell
                if (self.valid(row, col, number, board)):
                    board[row, col] = number
                else:
                    return False
        # solution is valid if all cells are valid
        return True

# Solving Sudoku Puzzle using Linear Optimization

Since sudoku only allows integer to be put into cells, a sudoku puzzle is therefore a mixed integer optimization problem.

However, unlike a normal linear optimization problem, the problem of sudoku puzzle does not have an objective function to be optimized. This is because as long as all the constraints are satisfied, a solution is considered valid. In other words, the objective function can be any arbitrary function which I will be using zero for simplicity. 

Now, let consider the decision variables.

Since a number must be unique in a row, a column, and a block, the decision variables cannot simply be each of the cells in the board. 

Instead, let utilize the structure of 9x9x9 grid where each cell is either 0 or 1 representing the existence of the corresponding value at the corresponding cell in the real sudoku board. This ways we can represent all the combination of the value, row, and column.

So, we have 

$$
x_{k, i, j} = 
\begin{cases}
    1 & \quad \text{if number $k$ is in row $i$, column $j$}, \\
    0 & \quad \text{otherwise}
\end{cases}
$$

Next, let define all the constraints.

1. Each row must contain the numbers without repetitions

$$
\sum_{j = 1}^{9} x_{k, i, j} = 1, \quad \quad \forall i, k \in [1, 9]
$$

2. Each column must contain the numbers without repetitions

$$
\sum_{i = 1}^{9} x_{k, i, j} = 1, \quad \quad \forall j, k \in [1, 9]
$$

3. Each block must contain the numbers without repetitiions

$$
\sum_{i = 1}^{9}\sum_{j = 1}^{9} x_{k, i + u, j + v} = 1, \quad \quad \forall k \in [1, 9] \text{ and } u, v \in \{3, 6, 9\}
$$

This is, however, still not enough because we also want to ensure that only one number can be in cell. So, we have 

4. Each cell must contain only one number

$$
\sum_{k = 1}^{9} x_{k, i, j} = 1, \quad \quad \forall i, j \in [1, 9]
$$

Also, since each of the cells in our 9x9x9 grid is either 0 or 1, we have 

5. Each cell in the 9 x 9 x 9 grid is either 0 or 1

$$
0 \leqslant x_{k, i, j} \leqslant 1, \quad \quad \forall i, j, k \in [1, 9]
$$ 

Finally, since we will use this to solve sudoku puzzles, we need to ensure that the position of the initial clues (i.e., the starting numbers that are given) are fixed. So, we have 

6. Each initial clue must be fixed

$$
x_{C_{i, j}, i, j} = 1, \quad \quad \forall C_{i, j}
$$

where $C_{i, j}$ is a starting number that is not 0 at row $i$ and column $j$

Now, let implement this using Python cvxpy

In [3]:
import cvxpy as cp

In [4]:
def solve_linear_optimization(self):
    x = { i : cp.Variable(shape = (self.n, self.n), integer = True) for i in range(self.n) }
    sqrt_n = (int) (np.sqrt(self.n))
    def get_constraints():
        # ensure a number can appear only once in a row/column
        row_constraints, col_constraints = map(list, zip(*[
            (cp.sum(x[k], axis = 1) == 1, cp.sum(x[k], axis = 0) == 1) 
            for k in range(self.n)
        ]))
        # ensure a number can appear only once in a sqrt(n)xsqrt(n) block
        block_constraints = []
        for k in range(self.n):
            for u in range(0, self.n - sqrt_n + 1, sqrt_n):
                for v in range(0, self.n - sqrt_n + 1, sqrt_n):
                    block_constraints.append(sum([
                        x[k][i + u, j + v] 
                        for i in range(sqrt_n) for j in range(sqrt_n)
                    ]) == 1)
        # ensure a number in the range and that every position is filled with only one number
        cell_constraints = list(sum([
            (0 <= x[k], x[k] <= 1) 
            for k in range(self.n)
        ], ())) + [
            sum([x[k][i, j] for k in range(self.n)]) == 1 
            for i in range(self.n) for j in range(self.n)
        ]
        # ensure the positions of known cells from the puzzle are fixed
        known_cell_constraints = [
            x[self.puzzle[i, j] - 1][i, j] == 1 
            for i in range(self.n) for j in range(self.n) 
            if (self.puzzle[i,j] != 0)
        ]
        return row_constraints + col_constraints + block_constraints + known_cell_constraints + cell_constraints
    def get_answer(variables):
        answer = np.copy(self.puzzle)
        for k, variable in enumerate(variables, start = 1):
            # find indexes where the cells are 1
            indexes = np.array(np.where(variable.value == 1))
            # for each 1, fill the corresponding cell in the answer with k
            for idx in range(len(indexes[0])):
                row, col = indexes[:, idx]
                if (answer[row, col] == 0):
                    answer[row, col] = k
        return answer
    # define problem
    prob = cp.Problem(
        # use zero as the objective function
        objective = cp.Minimize(0),
        # get and set all the constraints
        constraints = get_constraints()
    )
    # solve
    prob.solve(solver = cp.GLPK_MI)
    # return the answer
    return get_answer(prob.variables())

Sudoku.solve_linear_optimization = solve_linear_optimization

In [5]:
sudoku = Sudoku(n = 9)

print(sudoku.puzzle)

[[0 0 0 1 0 5 0 0 9]
 [0 2 9 0 0 7 0 5 8]
 [0 5 0 0 8 9 1 0 0]
 [0 0 3 0 4 0 8 9 0]
 [4 6 5 7 9 0 2 0 1]
 [0 7 8 0 1 0 0 4 6]
 [3 0 1 0 7 0 9 8 0]
 [0 0 0 0 0 0 7 0 0]
 [0 0 0 0 0 4 3 0 0]]


In [6]:
sol = sudoku.solve_linear_optimization()

print(sol)

print(sudoku.check(sol))

[[8 3 4 1 2 5 6 7 9]
 [1 2 9 3 6 7 4 5 8]
 [7 5 6 4 8 9 1 2 3]
 [2 1 3 5 4 6 8 9 7]
 [4 6 5 7 9 8 2 3 1]
 [9 7 8 2 1 3 5 4 6]
 [3 4 1 6 7 2 9 8 5]
 [5 9 2 8 3 1 7 6 4]
 [6 8 7 9 5 4 3 1 2]]
True


In [7]:
sols = []
sudoku.solve_backtracking(sudoku.puzzle, sols, need_one_solution = True)

print(sols[0])

print(sudoku.check(sols[0]))

[[8 3 4 1 2 5 6 7 9]
 [1 2 9 3 6 7 4 5 8]
 [6 5 7 4 8 9 1 2 3]
 [2 1 3 5 4 6 8 9 7]
 [4 6 5 7 9 8 2 3 1]
 [9 7 8 2 1 3 5 4 6]
 [3 4 1 6 7 2 9 8 5]
 [5 8 2 9 3 1 7 6 4]
 [7 9 6 8 5 4 3 1 2]]
True
